In [12]:
import pandas as pd

file_path = "/Users/mariapleon/Desktop/Data Science/all_top_songs_ful (dataset).csv"

# safer version for messy CSV files
df = pd.read_csv(file_path, sep=None, engine='python', encoding='utf-8', on_bad_lines='skip')

print(df.shape)
df.head()

(1252, 10)


,track_id,track_name,track_popularity,duration_ms,artist_names,artist_ids,album_id,album_name,album_release_date,album_total_tracks
0,5ZaDUleherOLBbHk2PU1Kr,Another Day in Paradise - 2016 Remaster,54,323040,Phil Collins,4lxfqrEsLX6N1N4OCSkILp,1t89giOgPycfALwe2NuVf8,...But Seriously (2016 Remaster),1989-11-20,12
1,3RMeOetCdXttthQK0clPuz,How Am I Supposed to Live Without You,70,255666,Michael Bolton,6YHEMoNPbcheiWS2haGzkn,5g9LXOhTPW9Iow6GZPRg2D,Soul Provider,1989-06-27,10
2,7z38bideBRvGAgjXe2SECm,Opposites Attract,51,265373,Paula Abdul,4PpmBoqphQusNFsxuVKb6j,7zduRJgS6v79QmNUhKGozu,Forever Your Girl,1988-01-01,10
3,5HAv1Ckfe50DUjv8ghwTrz,Escapade,55,283933,Janet Jackson,4qwGe91Bz9K2T8jXTZ815W,4OD3LU6001esAtFshDX46M,Rhythm Nation 1814,1989-09-19,20
4,1KU5EHSz04JhGg3rReGJ0N,Black Velvet,73,287440,Alannah Myles,6IYnSXO40Bh7Zdqhf6rQoj,1Ghv7iViywM23K8BRFggQv,Alannah Myles,1989-00-00,10


In [13]:
df.columns.tolist()


['track_id',
 'track_name',
 'track_popularity',
 'duration_ms',
 'artist_names',
 'artist_ids',
 'album_id',
 'album_name',
 'album_release_date',
 'album_total_tracks']

In [14]:
COLS = {
    "track_id":           "track_id",
    "track_name":         "track_name",
    "artist_names":       "artist_names",
    "album_release_date": "album_release_date",
    "genre":              None,                 # you don't have it (that's fine)
    "track_popularity":   "track_popularity",
}
COLS



{'track_id': 'track_id',
 'track_name': 'track_name',
 'artist_names': 'artist_names',
 'album_release_date': 'album_release_date',
 'genre': None,
 'track_popularity': 'track_popularity'}

In [15]:
import pandas as pd
import re

# Dates → proper datetime + helper year column
df[COLS["album_release_date"]] = pd.to_datetime(
    df[COLS["album_release_date"]], errors="coerce", utc=True
)
df["release_year"] = df[COLS["album_release_date"]].dt.year

# Artist names → strip extra spaces and Title Case
df[COLS["artist_names"]] = (
    df[COLS["artist_names"]]
      .astype(str)
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
      .str.title()
)



In [16]:
before = len(df)

# Pass A: exact duplicates by track_id
df = df.drop_duplicates(subset=[COLS["track_id"]], keep="first")
after_id = len(df)

# Pass B: collapse “same song” variants via normalized title + main artist
def normalize_title(name: str) -> str:
    s = str(name).lower()
    s = re.sub(r"[\(\[].*?[\)\]]", "", s)  # remove (Remastered…), [Live]…
    s = re.sub(r"\s*-\s*(remaster(ed)?(\s*\d{2,4})?|radio edit|live|mono|stereo|single version)\b","", s)
    s = re.sub(r"\s*(feat\.|featuring)\s+.+$", "", s)        # drop “feat …” tail
    return re.sub(r"\s+"," ", s).strip()

# Build normalized key
df["_tnorm"] = df[COLS["track_name"]].astype(str).apply(normalize_title)
df["_main_artist"] = df[COLS["artist_names"]].astype(str).str.split(r"[,&]").str[0].str.strip().str.title()
df["_dedupe_key"] = df["_main_artist"].str.lower() + " | " + df["_tnorm"]

# Prefer highest popularity (or earliest release if tie)
sort_order = [(COLS["track_popularity"], False), (COLS["album_release_date"], True)]
for col, asc in reversed(sort_order):
    df = df.sort_values(by=col, ascending=asc)

df = df.drop_duplicates(subset=["_dedupe_key"], keep="first")
df = df.drop(columns=["_tnorm","_main_artist","_dedupe_key"])

after_b = len(df)

print(f"Removed by track_id: {before - after_id}")
print(f"Removed by title+artist: {after_id - after_b}")
print("New shape:", df.shape)


Removed by track_id: 10
Removed by title+artist: 3
New shape: (1239, 11)


In [17]:
print(df.isna().sum().sort_values(ascending=False).head(10))
df.sample(3, random_state=7)


album_release_date    153
release_year          153
track_id                0
track_name              0
track_popularity        0
duration_ms             0
artist_names            0
artist_ids              0
album_id                0
album_name              0
dtype: int64


,track_id,track_name,track_popularity,duration_ms,artist_names,artist_ids,album_id,album_name,album_release_date,album_total_tracks,release_year
134,1Oi2zpmL81Q0yScF1zxaC0,"Bills, Bills, Bills",70,256026,Destiny'S Child,1Y8cdNmUJH7yBTd9yOvr5i,283NWqNsCA9GwVHrJk59CG,The Writing's On The Wall,1999-07-27 00:00:00+00:00,16,1999.0
315,5MnPcgXz4RXqx09nIqvHKX,Hello,74,155006,Cardi B,4kYSro6naA4h99UJvo89HB,1LsNb6mVitbLZCxnRsbCDJ,AM I THE DRAMA?,2025-09-19 00:00:00+00:00,23,2025.0
178,2tpWsVSb9UEmDRxAl1zhX1,Counting Stars,87,257265,Onerepublic,5Pwc4xIPtQLFEnJriah9YJ,20lOt6G8MHv8ZO7ViOmiP7,Native,2013-01-01 00:00:00+00:00,12,2013.0


In [18]:
out_path = "/Users/mariapleon/Desktop/Data Science/cleaned_top_songs.csv"
df.to_csv(out_path, index=False, encoding="utf-8")
print("Saved:", out_path)


Saved: /Users/mariapleon/Desktop/Data Science/cleaned_top_songs.csv


In [19]:
cols_order = [
    "track_id", "track_name", "artist_names",
    "album_id", "album_name", "album_release_date", "release_year",
    "artist_ids", "duration_ms", "album_total_tracks", "track_popularity"
]
df = df[[c for c in cols_order if c in df.columns]]
df.head(3)


,track_id,track_name,artist_names,album_id,album_name,album_release_date,release_year,artist_ids,duration_ms,album_total_tracks,track_popularity
240,1CPZ5BxNNd0n0nF4Orb9JS,Golden,Huntr/X; Ejae; Audrey Nuna; Rei Ami; Kpop Demo...,14JkAa6IiFaOh5s0nMyMU9,KPop Demon Hunters (Soundtrack from the Netfli...,2025-06-20 00:00:00+00:00,2025.0,2yNNYQBChuox9A5Ka93BIn; 0RMJOzHDhAKY1o2j0W0vxY...,194607,12,100
246,02sy7FAs8dkDNYsHp4Ul3f,Soda Pop,Saja Boys; Andrew Choi; Neckwav; Danny Chung; ...,14JkAa6IiFaOh5s0nMyMU9,KPop Demon Hunters (Soundtrack from the Netfli...,2025-06-20 00:00:00+00:00,2025.0,0BJ2EjOqcfgFvlZXNKrNbE; 7JBaynR7JFeniONqbnn1U4...,150686,12,95
249,1I37Zz2g3hk9eWxaNkj031,Your Idol,Saja Boys; Andrew Choi; Neckwav; Danny Chung; ...,14JkAa6IiFaOh5s0nMyMU9,KPop Demon Hunters (Soundtrack from the Netfli...,2025-06-20 00:00:00+00:00,2025.0,0BJ2EjOqcfgFvlZXNKrNbE; 7JBaynR7JFeniONqbnn1U4...,191537,12,95


In [20]:
df.isna().sum().sort_values(ascending=False)


album_release_date    153
release_year          153
track_id                0
track_name              0
artist_names            0
album_id                0
album_name              0
artist_ids              0
duration_ms             0
album_total_tracks      0
track_popularity        0
dtype: int64

In [21]:
df['album_release_date'] = df['album_release_date'].fillna('N/A')
df['release_year'] = df['release_year'].fillna('N/A')


In [22]:
df[['album_release_date', 'release_year']].head(10)
print(df.isna().sum().sort_values(ascending=False).head(10))


track_id              0
track_name            0
artist_names          0
album_id              0
album_name            0
album_release_date    0
release_year          0
artist_ids            0
duration_ms           0
album_total_tracks    0
dtype: int64


In [23]:
out_path = "/Users/mariapleon/Desktop/Data Science/cleaned_top_songs.csv"
df.to_csv(out_path, index=False, encoding="utf-8")
print("Saved:", out_path)


Saved: /Users/mariapleon/Desktop/Data Science/cleaned_top_songs.csv
